# 1.使用Pandas整理環保署PM2.5資料資料分析

In [1]:
import pandas as pd
df = pd.read_csv("https://data.epa.gov.tw/api/v2/aqx_p_02?api_key=c9c5dfbc-7a19-4452-96d6-f6b3f5396db3&limit=1000&sort=datacreationdate%20desc&format=csv")
print(df)

   site county  pm25  datacreationdate itemunit
0    大城    彰化縣    33  2023-04-16 19:00    μg/m3
1   富貴角    新北市    24  2023-04-16 19:00    μg/m3
2    麥寮    雲林縣    31  2023-04-16 19:00    μg/m3
3    關山    臺東縣    30  2023-04-16 19:00    μg/m3
4    馬公    澎湖縣    28  2023-04-16 19:00    μg/m3
..  ...    ...   ...               ...      ...
73   土城    新北市    28  2023-04-16 19:00    μg/m3
74   新店    新北市    25  2023-04-16 19:00    μg/m3
75   萬里    新北市    26  2023-04-16 19:00    μg/m3
76   汐止    新北市    28  2023-04-16 19:00    μg/m3
77   基隆    基隆市    32  2023-04-16 19:00    μg/m3

[78 rows x 5 columns]


# 2.選項按鈕處理

In [1]:
def rbCity():  #點選縣市選項按鈕後處理函式
    global sitelist, listradio
    sitelist.clear()  #清除原有測站串列
    for r in listradio:  #移除原有測站選項按鈕
        r.destroy()
    n=0
    for c1 in data["county"]:  #逐一取出選取縣市的測站
        if(c1 == city.get()):
            site1 = data.iloc[n, 0]
            if(site1 not in sitelist):
                sitelist.append(site1)
        n += 1    
    sitemake()  #建立測站選項按鈕
    rbSite()  #顯示PM2.5訊息

def rbSite():  #點選測站選項按鈕後處理函式
    n = 0
    for s in data.iloc[:, 0]:  #逐一取得測站
        if(s == site.get()):  #取得點選的測站
            pm = data.iloc[n, 2]  #取得PM2.5的值
            if(pm=='' or pm=='ND'):  #如果沒有資料
                result1.set(s + "站的 PM2.5 值目前無資料！")
            else:  #如果有資料
                if(int(pm) <= 35):  #轉換為等級
                    grade1 = "低"
                elif(int(pm) <= 53):
                    grade1 = "中"
                elif(int(pm) <= 70):
                    grade1 = "高"
                else:
                    grade1 = "非常高"
                result1.set(s + "站的 PM2.5 值為「" + str(pm) + "」：「" + grade1 + "」等級")
            break  #找到點選測站就離開迴圈
        n += 1
    
def clickRefresh():  #重新讀取資料
    global data
    data = pd.read_csv("https://data.epa.gov.tw/api/v2/aqx_p_02?api_key=c9c5dfbc-7a19-4452-96d6-f6b3f5396db3&limit=1000&sort=datacreationdate%20desc&format=csv")
    rbSite()  #更新測站資料

def sitemake():  #建立測站選項按鈕
    global sitelist, listradio
    for c1 in sitelist:  #逐一建立選項按鈕
        rbtem = tk.Radiobutton(frame2, text=c1, variable=site, value=c1, command=rbSite)  #建立選項按鈕
        listradio.append(rbtem)  #加入選項按鈕串列
        if(c1==sitelist[0]):  #預設選取第1個項目         
            rbtem.select()
        rbtem.pack(side="left")  #靠左排列

import tkinter as tk
import pandas as pd

data = pd.read_csv("https://data.epa.gov.tw/api/v2/aqx_p_02?api_key=c9c5dfbc-7a19-4452-96d6-f6b3f5396db3&limit=1000&sort=datacreationdate%20desc&format=csv")

win=tk.Tk()
win.geometry("800x270")
win.title("PM2.5 實時監測")

city = tk.StringVar()  #縣市文字變數
site = tk.StringVar()  #測站文字變數
result1 = tk.StringVar()  #訊息文字變數
citylist = []  #縣市串列
sitelist = []  #鄉鎮串列
listradio = []  #鄉鎮選項按鈕串列

#建立縣市串列
for c1 in data["county"]:  
    if(c1 not in citylist):  #如果串列中無該縣市就將其加入
        citylist.append(c1)
#建立第1個縣市的測站串列
count = 0
for c1 in data["county"]:  
    if(c1 ==  citylist[0]):  #是第1個縣市的測站
        site1 = data.iloc[count, 0]
        if(site1 not in sitelist):
            sitelist.append(site1)
    count += 1

label1 = tk.Label(win, text="縣市：", pady=8, fg="blue", font=("新細明體", 16))
label1.pack()
frame1 = tk.Frame(win)  #縣市容器
frame1.pack()
for i in range(0,3):  #3列選項按鈕
    for j in range(0,8):  #每列8個選項按鈕
        n = i * 8 + j  #第n個選項按鈕
        if(n < len(citylist)):
            city1 = citylist[n]  #取得縣市名稱
            rbtem = tk.Radiobutton(frame1, text=city1, variable=city, value=city1, command=rbCity)  #建立選項按鈕
            rbtem.grid(row=i, column=j)  #設定選項按鈕位置
            if(n==0):  #選取第1個縣市
                rbtem.select()

label2 = tk.Label(win, text="測站：", pady=8, fg="blue", font=("新細明體", 16))
label2.pack()
frame2 = tk.Frame(win)  #測站容器
frame2.pack()
sitemake()

btnDown = tk.Button(win, text="更新資料", font=("新細明體", 16), command=clickRefresh)
btnDown.pack(pady=6)
lblResult1 = tk.Label(win, textvariable=result1, fg="red", font=("新細明體", 16))
lblResult1.pack(pady=6)
rbSite()  #顯示測站訊息

win.mainloop()